In [1]:
# Import libraries
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from tokenizers.decoders import WordPiece

# Load model
model_name = "avichr/heBERT_NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

oracle = pipeline('ner', model='dicta-il/dictabert-ner', aggregation_strategy='simple')
oracle.tokenizer.backend_tokenizer.decoder = WordPiece()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cpu


In [2]:
# Load uploaded file
df = pd.read_csv("/content/AlephBERT_predictions.csv")

In [3]:

# Define intent templates
intent_templates = {
    "transport_query": "את יכולה למצוא לי מסלול {origin} {destination} ב{mode}?",
    "alarm_set": "את יכולה לכוון שעון מעורר לשעה {time} {period}",
    "call_contact": "את יכולה לחייג ל{contact_name}?",
    "send_message": "את יכולה לשלוח הודעה ל{contact_name} {message}",
    "calendar_set": "את יכולה ליצור לי פגישה ביומן ל{date} ב{time} בשם {meeting_title}?",
    "camera_query": "אני רוצה לצלם {type}",
    "lists_createoradd": "תיצרי לי פתק חדש שיהיה כתוב בו {items}",
    "weather_query": "מה מזג האוויר {date} {location}?",
    "iot_wemo_on": "אני רוצה להדליק {device}",
    "query": "את יכולה לבדוק לי {search string}"
}


# Extract keys from templates
intent_keys = {
    intent: re.findall(r"{(.*?)}", template)
    for intent, template in intent_templates.items()
}


# Helper functions
def extract_entities(ner_result, label):
    return [entity["word"] for entity in ner_result if label in entity["entity_group"]]

def extract_entity(ner_result, label):
    entities = extract_entities(ner_result, label)
    return entities[0] if entities else None


In [4]:
from google.colab import files

def extract_destination_fallback(transcript):
    # טיפול מיוחד בביטויים שמתחילים ב"תפתחי לי ב[maps|מפות] ..."
    match = re.search(r"תפתחי(?:\s+לי)?\s+ב(?:maps|מפות)?\s+(.*)", transcript)
    if match:
        possible_address = match.group(1).strip()
        # נסה למצוא כתובת בתוך מה שנשאר
        addr_match = re.search(r"(שדרות|רחוב|דרך)\s+[א-ת׳\"״]+\s*\d*\s*(?:ב[א-ת\s]+)?", possible_address)
        if addr_match:
            return addr_match.group().strip()
        return possible_address

    # טיפול בביטוי "עם הכתובת"
    match = re.search(r"(?:לפתוח\s+)?(?:גוגל\s+)?מפס\s+עם\s+הכתובת\s+(.+)", transcript)
    if match:
        return match.group(1).strip()

    # ניסיון לזהות כתובת בפורמט סטנדרטי
    street_pattern = re.search(r"(שדרות|רחוב|דרך)\s+[א-ת\"׳״]+\s*\d*\s*ב[א-ת\"׳״\s]+", transcript)
    if street_pattern:
        return street_pattern.group().strip()

    # fallback רגיל
    ignored_starters = {
        "לקבל", "לקבל הסבר", "לקבל מידע", "לקבל עזרה",
        "לנווט", "לכוון", "להגיע", "ללכת", "לנסוע",
        "תעזור", "תעזרי", "תכווני", "תכוון", "לפתוח", "גוגל", "מפס",
        "maps", "עם", "הכתובת", "תפתחי", "במפות", "ב־maps", "לי"
    }

    stopwords = {
        "ש", "אם", "אז", "אבל", "ואם", "או", "על", "של", "עם", "שבו", "ב", "לב", "ול",
        "האם", "איך", "תגידי", "תגיד", "תגידו", "תוכלי", "תוכל", "לעזור", "לי", "בבקשה",
        "אתה", "אני", "אולי", "תעזרי", "תעזור", "תכוון", "תכווני", "מפה", "צריכה", "צריך", "סימון"
    }

    for phrase in ignored_starters:
        transcript = transcript.replace(phrase, "")

    words = transcript.strip().split()
    bad_starts = {"מפה", "תמונה", "סימון", "תצוגה"}
    while words and words[0] in bad_starts:
        words = words[1:]

    collecting = False
    dest_words = []

    for i, word in enumerate(words):
        stripped = word.strip(".,!?")
        if not collecting and (stripped.startswith("ל") and len(stripped) > 3):
            collecting = True
            dest_words = [stripped]
            continue
        if collecting:
            if stripped in stopwords:
                break
            dest_words.append(stripped)

    if not dest_words and len(words) >= 2:
        candidate = []
        for word in words:
            w = word.strip(".,!?")
            if w in stopwords:
                break
            candidate.append(w)
        dest_words = candidate

    while dest_words and dest_words[-1] in {"ב", "ל", "של", "על", "מפה"}:
        dest_words.pop()

    return " ".join(dest_words)


def clean_destination_of_mode(destination, mode):
    """Remove mode word from the end of the destination, if it appears."""
    if destination.endswith(mode):
        return destination[:-len(mode)].strip()
    return destination

def clean_destination_suffix(destination):
    # הסר תווים חריגים מסוף הכתובת
    destination = destination.strip(".,!? ").strip()

    # הסר מילה בודדת לא לגיטימית בסוף (למשל "ב", "ל", "של", "על")
    if re.search(r"\b(ב|ל|של|על)$", destination):
        destination = re.sub(r"\b(ב|ל|של|על)$", "", destination).strip()
    return destination



def build_action_json(transcript, ner_result, intent):
    intent = intent.strip()
    keys = intent_keys.get(intent, [])
    result = {"intent": intent}

    # Extract entities
    full_name = extract_entity(ner_result, "PER")

    if "contact_name" in keys and full_name:
        # ניקוי תווים לא רצויים
        full_name = full_name.replace("##", "").strip()

        # אם מתחיל ב"ל" ונשמע כמו שם (למשל: ליוני, לדנה) – הסר את "ל"
        if full_name.startswith("ל") and len(full_name) > 2:
            full_name = full_name[1:]

        result["contact_name"] = full_name


    location = extract_entity(ner_result, "LOC")
    date = extract_entity(ner_result, "DATE")
    time = extract_entity(ner_result, "TIME")
    locs = extract_entities(ner_result, "LOC")
    msg = transcript.split(":")[-1] if ":" in transcript else transcript.split(full_name)[-1].strip() if full_name and full_name in transcript else None


    for key in keys:
        if intent == "transport_query":
            result["mode"] = "הליכה"
            result["destination"] = ""
            result["origin"] = "מהמיקום שלי"

            modes = {
                ("רכבת", "אוטובוס"): "תחבורה ציבורית",
                ("רגל", "הלכ"): "הליכה",
                ("רכב", "ברכב", "אוטו", "נהיגה", "לנסוע"): "רכב"
            }

            for mode in modes:
                for transport in mode:
                    regex = rf"{transport}"
                    if re.search(regex, transcript):
                        result["mode"] = modes[mode]
                        continue

            ner_result = oracle(transcript)

            allowed_ents = ["GPE", "FAC", "PER", "CARDINAL", "NUMBER", "LOC"]
            to_regex = r"^ל"
            from_regex = r"^מ"

            prev_state = None
            destination_words = []
            origin_words = []

            # מילים שמתחילות ב"ל" אבל אינן כתובות, אלא פעלים כלליים
            invalid_to_words = {
                "להגיע", "להראות", "לנסוע", "לפתוח", "לכוון", "לחפש", "לבדוק",
                "לשאול", "לראות", "לשמור", "להתקשר", "לשלוח", "לנווט"
            }

            for res in ner_result:
                extracted_word = transcript[res['start']:min(res['end'] + 1, len(transcript))].strip()
                entity = res['entity_group']

                if entity in allowed_ents:
                    if re.search(to_regex, extracted_word) and not destination_words:
                        if extracted_word in invalid_to_words:
                            continue  # התעלם מהמילה הזו — זה פועל, לא יעד
                        destination_words.append(extracted_word)
                        prev_state = "destination"
                    elif re.search(from_regex, extracted_word) and not origin_words:
                        origin_words.append(extracted_word)
                        prev_state = "origin"
                    elif prev_state == "destination":
                        destination_words.append(extracted_word)
                    elif prev_state == "origin":
                        origin_words.append(extracted_word)
                else:
                    prev_state = None


            if destination_words:
                result["destination"] = " ".join(destination_words)
                result["destination"] = clean_destination_of_mode(result["destination"], result["mode"])

                result["destination"] = clean_destination_suffix(result["destination"])

            if origin_words:
                result["origin"] = " ".join(origin_words) or "מהמיקום שלי"

            # Fallback destination if NER failed
            if not result["destination"]:
                fallback_address = extract_destination_fallback(transcript)
                if fallback_address:
                    result["destination"] = clean_destination_of_mode(fallback_address, result["mode"])
                    result["destination"] = clean_destination_suffix(result["destination"])


        elif key == "period":
            if any(p in transcript for p in ["בבוקר", "בוקר"]):
                result[key] = "בבוקר"
            elif any(p in transcript for p in ["בערב", "לערב", "ערב"]):
                result[key] = "בערב"
            elif any(p in transcript for p in ["בלילה", "ללילה"]):
                result[key] = "בלילה"
            elif "אחר הצהריים" in transcript or "בצהריים" in transcript:
                result[key] = "בצהריים"
            elif "מחר" in transcript:
                result[key] = "מחר"
            elif "היום" in transcript:
                result[key] = "היום"
            elif time:
                try:
                    hour = int(time.split(":")[0])
                    if 5 <= hour < 12:
                        result[key] = "בבוקר"
                    elif 12 <= hour < 17:
                        result[key] = "בצהריים"
                    else:
                        result[key] = "בערב"
                except:
                    result[key] = None
            else:
                result[key] = None


        elif key == "contact_name":
            if full_name:
                clean_name = full_name.strip()
                # טיפול בכפילות של מילים עוקבות (כמו "אנה אנה לוי")
                words = clean_name.split()
                deduped = []
                for i, word in enumerate(words):
                    if i == 0 or word != words[i - 1]:
                        deduped.append(word)
                result[key] = " ".join(deduped)
            else:
                match = re.search(r"ל([א-ת]+(?: [א-ת]+)?)", transcript)
                if match:
                    name = match.group(1).strip()
                    result[key] = name
                else:
                    result[key] = None


        elif key == "message":
            msg = None

            # אוסף ביטויים טיפוסיים של הודעות
            common_phrases = [
                r"תתקשר(י|י)? אליי כשתתפנה(?: בבקשה)?",
                r"תתקשר אלי כשתתפנה(?: בבקשה)?",
                r"שיתקשר אליי כשהוא מתפנה",
                r"שיתקשר אלי כשהוא מתפנה",
                r"שיתקשר אליי כשיהיה פנוי",
                r"שיתקשר אלי כשיהיה פנוי",
                r"ש?הוא יתקשר אליי כשהוא מתפנה",
                r"ש?הוא יתקשר אלי כשהוא מתפנה",
                r"ש?הוא יתקשר אליי כשיהיה פנוי",
                r"ש?הוא יתקשר אלי כשיהיה פנוי"
            ]

            for phrase in common_phrases:
                match = re.search(phrase, transcript)
                if match:
                    msg = match.group().strip()
                    break

            if not msg:
                # fallback: טקסט אחרי 'באס אם אס'
                match = re.search(r"באס[ -]?א?ם[ -]?א?ס\s+(.*)", transcript)
                if match:
                    msg = match.group(1).strip()

            # ניקוי מילים מיותרות
            if msg:
                garbage_patterns = [
                    r"ה?ה?ודעת( ה)?טקסט( הבאה)?(?:\s+בבקשה)?(?:\s+תשלחי\s+לו)?",
                    r"באס ?אם ?אס",
                    r"סמס", r"SMS", r"ב[- ]?SMS",
                    r"\bבבקשה\b", r"\bאת\b", r"\bהבאה\b", r"\bב\b",
                    r"תשלחי לו", r"אנא שלחי", r"שלחי",
                    r"לישראל זילברמן", r"לאלי זילברמן"
                ]

                cleaned_msg = msg
                for pattern in garbage_patterns:
                    cleaned_msg = re.sub(pattern, "", cleaned_msg, flags=re.IGNORECASE)

                cleaned_msg = re.sub(r"\s{2,}", " ", cleaned_msg).strip()
                result[key] = cleaned_msg if cleaned_msg else None
            else:
                result[key] = None



        elif key == "meeting_title":
            title = None

            # חיפוש ביטויים שמסמנים שם פגישה
            patterns = [
                r"(?:בשם|שמה הפגישה תהיה|ותקראי ל[הו]?|תני שם לפגישה|תקראי ל[הו]?)\s+(.+?)(?=\s+(?:ש[הי]וא|פגישה|עם|בתאריך|בשעה|ברביעי|בחמישי|ב[\w]+)|$)",
                r"פגישה בשם\s+(.+?)(?=\s+(?:ש[הי]וא|עם|בתאריך|בשעה|ברביעי|בחמישי|ב[\w]+)|$)"
            ]

            for pattern in patterns:
                match = re.search(pattern, transcript)
                if match:
                    title = match.group(1).strip(" .,")
                    break

            result[key] = title if title else "פגישה"



        elif key == "date":
            if intent == "weather_query":
                result[key] = date or "היום"
            else:
                date_value = None

                # שלב 1: תאריך במספרים (למשל 04/06/2025)
                date_match = re.search(r"\b(\d{1,2})[./](\d{1,2})[./](\d{2,4})\b", transcript)
                if date_match:
                    day, month, year = date_match.groups()
                    date_value = f"{int(day):02}/{int(month):02}/{year}"

                # שלב 2: תאריך מילולי (למשל "ברביעי לשישי", "בראשון ביולי")
                if not date_value:
                    verbal_match = re.search(
                        r"(?:בתאריך\s*)?(?:ב)?(ראשון|שני|שלישי|רביעי|חמישי|שישי|שבת)\s*(?:ל|ב)([א-ת]+|\d{1,2})(?:\s*(\d{2,4}))?",
                        transcript
                    )
                    if verbal_match:
                        weekday, month, year = verbal_match.groups()

                        # מיפוי של שמות חודשים עבריים ממוספרים למילוליים
                        hebrew_month_map = {
                            "ראשון": "ינואר",
                            "שני": "פברואר",
                            "שלישי": "מרץ",
                            "רביעי": "אפריל",
                            "חמישי": "מאי",
                            "שישי": "יוני",
                            "שביעי": "יולי",
                            "שמיני": "אוגוסט",
                            "תשיעי": "ספטמבר",
                            "עשירי": "אוקטובר",
                            "אחד עשר": "נובמבר",
                            "שתים עשרה": "דצמבר"
                        }

                        # ניקוי הקידומת ("ל", "ב") מהמילה כדי לבדוק במפה
                        month_clean = re.sub(r"^(ל|ב)", "", month)
                        month_normalized = hebrew_month_map.get(month_clean, month_clean)

                        parts = [weekday]
                        parts.append("ל" + month_normalized)
                        if year:
                            parts.append(year)
                        date_value = " ".join(parts).strip()

                # שלב 3: Fallback למידע מ-NER אם יש משהו שימושי
                if not date_value and date and date.strip() not in {"בתאריך", "לתאריך"}:
                    date_value = date.strip()

                result[key] = date_value or "היום"

        elif key == "time":
            time_candidate = None

            # שלב 1: פורמט רגיל - 6:48, 06.30 וכו'
            time_match = re.search(r"(\d{1,2})[:٫.](\d{2})", transcript)
            if time_match:
                result[key] = f"{int(time_match.group(1)):02}:{time_match.group(2)}"
                continue

            # # שלב 2: לשעה / בשעה
            # word_time_match = re.search(r"(?:לשעה|בשעה)\s+([א-ת\s]+)", transcript)
            # if word_time_match:
            #     raw_time = word_time_match.group(1).strip()

            #     # חיתוך במילה שמתחילה ב"ל" ולא נראית כמו חלק משעה (למשל: "לדוקטור")
            #     tokens = raw_time.split()
            #     time_tokens = []
            #     for token in tokens:
            #         if re.match(r"^ל(?!שעה)", token) or token in {"בבקשה", "תודה"}:
            #             break
            #         time_tokens.append(token)

            #     time_candidate = " ".join(time_tokens).strip()

            # שלב 2: לשעה / בשעה
            word_time_match = re.search(r"(?:לשעה|בשעה)\s+([א-ת\s]+)", transcript)
            if word_time_match:
                raw_time = word_time_match.group(1).strip()

                # עצירה במילים שמבשרות על ישויות אחרות (contact, title וכו')
                stop_words = {
                    "בשם", "תקראי", "תקרא", "תור", "עם", "לדוקטור", "לד״ר", "פגישה", "לרופא", "הרופא"
                }

                tokens = raw_time.split()
                time_tokens = []
                for token in tokens:
                    if token in stop_words or token.startswith("ל") and token not in {"לחמש", "לאחת", "לשתיים"}:
                        break
                    time_tokens.append(token)

                time_candidate = " ".join(time_tokens).strip()



            # שלב 3: אם intent == alarm_set, חפש אחרי "ל"/"ב" ודלג על "בקשה"
            elif intent == "alarm_set":
                words = transcript.split()
                for i, word in enumerate(words):
                    if word in {"ב", "ל"} and i + 1 < len(words):
                        next_index = i + 1
                        if words[next_index] == "בקשה":
                            next_index += 1
                        if next_index < len(words):
                            time_candidate = " ".join(words[next_index:])
                            break

            # שלב 4: fallback – חיפוש רצף של מילים שמזכירות מספרים
            if not time_candidate:
                matches = re.findall(
                    r"(שבע|שש|שמונה|חמש|ארבע|תשע|עשר|עשרים|שלושים|חמישים|ארבעים)"
                    r"(?:\s+(אחת|שתיים|שלוש|שש|שבע|שמונה|תשע|עשר|עשרים|שלושים|חמישים|ארבעים)*)?",
                    transcript
                )
                if matches:
                    time_candidate = " ".join([m[0] + (" " + m[1] if m[1] else "") for m in matches])

            # שלב 5: זיהוי ביטויים כמו "שלוש ורבע", "שתיים וחצי", "אחת בדיוק"
            if not time_candidate:
                time_words_match = re.search(
                    r"(?:שעה\s+)?(אחת|שתיים|שלוש|ארבע|חמש|שש|שבע|שמונה|תשע|עשר)"
                    r"(?:\s+(ורבע|וחצי|בדיוק))?",
                    transcript
                )
                if time_words_match:
                    time_candidate = " ".join([g for g in time_words_match.groups() if g])


            # ניקוי מילים מיותרות
            if time_candidate:
                # ניקוי ביטויים שלמים קודם
                phrases_to_remove = [
                    "תודה מראש", "תודה רבה", "אני רוצה", "אני צריכה", "אשמח אם", "אפשר לכוון"
                ]
                for phrase in phrases_to_remove:
                    time_candidate = time_candidate.replace(phrase, "")

                # ניקוי מילים בודדות אחר כך
                garbage = [
                    "תודה", "מראש", "בבקשה", "בכיף",
                    "תוכלי", "תוכל", "תכווני", "לכוון", "לכיוון",
                    "את", "אני", "מוכנה", "מעורר", "שעון", "השעון", "השעה"
                ]
                for word in garbage:
                    time_candidate = re.sub(rf"\b{word}\b", "", time_candidate)

                # ניקוי רווחים
                time_candidate = re.sub(r"\s{2,}", " ", time_candidate).strip()

                if len(time_candidate.split()) >= 1:
                    result[key] = time_candidate
                else:
                    result[key] = None
            else:
                result[key] = None


        elif key == "items":
            raw_items = transcript

            # מילות מפתח שמסמנות את תחילת רשימת הפריטים
            item_starters = {"חלב", "ביצים", "דגים", "מלח", "שמן", "לחם", "עגבניות", "מים", "שוקולד", "אורז", "פסטה"}

            # חפש את מיקום ההתחלה של מילה כזו
            words = raw_items.split()
            start_index = next((i for i, word in enumerate(words) if word.strip(" ,.") in item_starters), None)

            if start_index is not None:
                items_raw = " ".join(words[start_index:])
            else:
                # fallback אם לא מצאנו מילה ברורה, נשתמש בגרסה הישנה עם סינון
                items_raw = raw_items

            # סינון ביטויים כלליים
            garbage_phrases = [
                "אני רוצה", "אני צריכה", "אני צריך", "תוסיפי לי", "תוסיפי", "תכיני לי",
                "בבקשה", "רשימת קניות", "רשימה", "פתק", "הפתק", "של", "שיהיה כתוב בו",
                "בפתק", "הפתק יקרא", "יהיה רשום", "שורה למטה", "רשום בו", "כתוב בו",
                "את יכולה", "לפתוח לי", "תפתחי לי", "תיצרי לי", "תכיני", "לי", "לי ל",
                "בפתקים", "לפתקים", "הקניות", "הפתק", "פתק חדש"
            ]

            for phrase in sorted(garbage_phrases, key=len, reverse=True):
                items_raw = items_raw.replace(phrase, "")

            # ניקוי כפילויות
            tokens = items_raw.strip().split()
            cleaned_tokens = [t.strip(",. ") for t in tokens if len(t) > 1]
            cleaned = " ".join(cleaned_tokens)

            # פיצול לרשימה והסרת כפילויות
            parts = re.split(r"\s+וגם\s+|\s+ו\s+|,|\s+ו(?=\S)", cleaned)
            unique_cleaned = list(dict.fromkeys([p.strip(" ,.-") for p in parts if len(p.strip(" ,.-")) > 1]))

            # בניית טקסט סופי
            if len(unique_cleaned) > 1:
                result[key] = ", ".join(unique_cleaned[:-1]) + " ו" + unique_cleaned[-1]
            elif unique_cleaned:
                result[key] = unique_cleaned[0]
            else:
                result[key] = None



        elif key == "type":
            if "סלפי" in transcript or re.search(r"\bה?מצלמה( הקדמית)?\b", transcript):
                result[key] = "סלפי"
            else:
                result[key] = "תמונה"

        elif key == "location":
            result[key] = location or ("תל אביב" if "תל אביב" in transcript else None)

        elif key == "device":
            known_devices = ["פנס", "מצלמה", "מצב טיסה", "wifi", "בלוטות", "טעינה"]
            for dev in known_devices:
                if dev in transcript:
                    result[key] = dev
                    break
            else:
                result[key] = "התקן"


        # elif key == "search string":
        #     raw = transcript

        #     # מחיקה של ביטויים כלליים ופסולת שפתית
        #     garbage_phrases = [
        #         "אני רוצה", "אני מעוניינת","מבקשת", "אני מבקשת", "הייתי רוצה", "בבקשה",
        #         "תודה מראש", "תודה רבה","להיות", "תודה", "לשאול", "לדעת", "לבדוק",
        #         "שם", "מהו", "מה הוא", "מהו שם", "של ה", "תגידי", "תגיד", "תגידו"
        #     ]
        #     for phrase in garbage_phrases:
        #         raw = raw.replace(phrase, "")

        #     # הסרת מילים בודדות מיותרות כמו "אני" אם נשארה
        #     garbage_words = ["אני", "תודה", "להיות"]
        #     for word in garbage_words:
        #         raw = re.sub(rf"\b{word}\b", "", raw)

        #     # ניקוי רווחים כפולים והוספה של סימן שאלה בסוף
        #     cleaned = re.sub(r"\s{2,}", " ", raw).strip()
        #     if not cleaned.endswith("?"):
        #         cleaned += "?"
        #     result[key] = cleaned

        elif key == "search string":
            raw = transcript.strip()

            # שלב 1: מחיקה של פתיחים שמופיעים רק בתחילת המשפט
            garbage_prefixes = [
                "את יכולה לבדוק לי", "את יכולה לבדוק", "את יכולה", "תוכל לבדוק", "תוכלי לבדוק",
                "תגידי לי", "תגיד לי", "תגידו לי", "אני רוצה לדעת", "אני מעוניינת לדעת",
                "אני מבקש לדעת", "אני רוצה לשאול", "אפשר לדעת", "אני מעוניין לדעת"
            ]
            for phrase in sorted(garbage_prefixes, key=len, reverse=True):
                if raw.startswith(phrase):
                    raw = raw[len(phrase):].strip()

            # שלב 2: המשך הקוד המקורי שלך — שמרנו עליו
            garbage_phrases = [
                "אני רוצה", "אני מעוניינת","מבקשת", "אני מבקשת", "הייתי רוצה", "בבקשה",
                "תודה מראש", "תודה רבה","להיות", "תודה", "לשאול", "לדעת", "לבדוק",
                "שם", "מהו", "מה הוא", "מהו שם", "של ה", "תגידי", "תגיד", "תגידו"
            ]
            for phrase in garbage_phrases:
                raw = raw.replace(phrase, "")

            garbage_words = ["אני", "תודה", "להיות"]
            for word in garbage_words:
                raw = re.sub(rf"\b{word}\b", "", raw)

            # שלב 3: זיהוי מילת שאלה והתחלה ממנה (אופציונלי – לא אגרסיבי מדי)
            question_starters = ["מהי", "מה", "מי", "כמה", "מתי", "איך", "איפה"]
            for q_word in question_starters:
                if q_word in raw:
                    raw = raw[raw.index(q_word):]
                    break

            # ניקוי רווחים וסיום
            cleaned = re.sub(r"\s{2,}", " ", raw).strip()
            if cleaned and not cleaned.endswith("?"):
                cleaned += "?"

            result[key] = cleaned


    return result

# Extract intent_json
intent_jsons = []
for index, row in df.iterrows():
    transcript = row["command"]
    intent = row["predicted_intent"]
    try:
        ner_result = ner_pipeline(transcript)
        json_obj = build_action_json(transcript, ner_result, intent)
        if intent == "call_contact":
            contact = json_obj.get("contact_name", "")
            # הסר רק "ל" בודדת שמופיעה לפני שם, לא כחלק מהמילה
            contact = re.sub(r"^ל(?=\s*[א-ת])", "", contact).strip()
            json_obj["contact_name"] = contact

    except Exception as e:
        json_obj = None

    intent_jsons.append(json_obj)

df["intent_json"] = intent_jsons

# Save results
df.to_csv("intent_extracted_cleaned.csv", index=False, encoding="utf-8-sig")
# files.download("intent_extracted_cleaned.csv")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
def rephrase_from_intent_json(intent, intent_json):
    template = intent_templates.get(intent)
    if not template or not intent_json:
        return None

    # Rename keys if needed
    mapping = {
        "search": "search string"
    }

    for old_key, new_key in mapping.items():
        if old_key in intent_json and new_key not in intent_json:
            intent_json[new_key] = intent_json[old_key]

    # החלפה של None בערכים ריקים (שיהפכו ל־"" במילוי התבנית)
    filled = {
        k: ("" if v is None else v)
        for k, v in intent_json.items()
    }

    # ניקוי כפילות של period מתוך time (למשל: "תשע ארבעים בערב" + "בערב")
    if intent == "alarm_set":
        time_val = filled.get("time", "")
        period_val = filled.get("period", "")
        if time_val and period_val and period_val in time_val:
            filled["period"] = ""  # אל תוסיף period שוב

    try:
        rephrased = template.format(**filled).strip()
        rephrased = re.sub(r"\s{2,}", " ", rephrased)  # הסרת רווחים כפולים
        return rephrased
    except Exception as e:
        return f"שגיאה בפורמט: {e}"


# Create a new column with rephrased sentence
df["rephrased"] = df.apply(lambda row: rephrase_from_intent_json(row["predicted_intent"], row["intent_json"]), axis=1)


# Save the new CSV including rephrased column
df.to_csv("rephrased_intents.csv", index=False, encoding="utf-8-sig")
files.download("rephrased_intents.csv")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>